In [1]:
import wikipedia
import nltk
import re
from pymystem3 import Mystem
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# for pretty output
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [3]:
pd.set_option('max_colwidth', 80)
wikipedia.set_lang('ru')
tokenizer = nltk.RegexpTokenizer(r"\w+")
mystem = Mystem()

Факты

In [4]:
facts = [
    "Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.",
    "Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их тщательно дезинфицировали.",
    "В России акции протеста проходят не только на площадях, но и на поездах."
]

Название статей Википедии, на которые ссылаются факты

In [5]:
wikipedia_pages = {
    1: ["Шмидт, Генрих (врач)"],
    2: ["Ходоки у В. И. Ленина", "Дезинфекция"],
    3: ["Проезд снаружи поездов"]
}

### Скачивание и обработка статей из Википедии

In [6]:
def get_page_text(pagename):
    page = wikipedia.page(pagename)
    return page.content

In [7]:
def remove_trash(page_text):
    page_text = re.sub('\s', ' ', page_text)
    page_text = re.sub('[А-Я]\.', '', page_text)
    page_text = re.sub(r'\=\= Примечания[\w\s\=]*', '', page_text)
    page_text = page_text.replace('.', '. ')
    page_text = page_text.replace('\n', ' ')
    headers = re.findall(r"\=\=.*?\=\=", page_text)
    for header in headers:
        page_text = page_text.replace(header, '')
    
    return page_text

In [8]:
documents = []

for num in wikipedia_pages.keys():
    for page in wikipedia_pages[num]:
        page_text = remove_trash(get_page_text(page))
        sentences = nltk.sent_tokenize(page_text, language="russian")
        documents.extend(sentences)

In [9]:
# добавление фактов
documents.extend(facts)

In [10]:
wiki = pd.DataFrame(documents)

In [11]:
wiki.columns = ['document']

In [12]:
wiki.head()

,document
0,"Эрнст Генрих Шмидт (нем. Ernst Heinrich Schmidt; 27 марта 1912, Альтенбург,..."
1,Генрих Шмидт родился 27 марта 1912 года в Альтенбурге.
2,В 1933 году был зачислен в СС (№ 23 069).
3,В 1937 году получил степень доктора медицинских наук в Лейпцигском университ...
4,После начала Второй мировой войны работал в лазарете Ваффен С С 1941 года с...


### Лемматизация 

In [13]:
def lemmatize(sentence):
    return ''.join((mystem.lemmatize(' '.join(tokenizer.tokenize(sentence))))).replace('\n', '')

In [14]:
wiki['lemmatized_document'] = wiki.document.apply(lemmatize)

In [15]:
wiki.tail()

,document,lemmatized_document
561,Дата обращения 7 декабря 2012.,дата обращение 7 декабрь 2012
562,Архивировано 27 декабря 2012 года.,архивировать 27 декабрь 2012 год
563,"Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправда...",нацистский врач эсэсовец работать в шесть концлагерь быть дважды оправдывать...
564,"Историк утверждает, что, прежде чем допустить крестьян на личный прием к вож...",историк утверждать что прежде чем допускать крестьянин на личный прием к вож...
565,"В России акции протеста проходят не только на площадях, но и на поездах.",в россия акция протест проходить не только на площадь но и на поезд


In [16]:
def tf_idf_vectorizer(corpus):
    
    tf = []
    
    vocabulary = np.unique([word for word in ' '.join(corpus).split(' ') if len(word) != 0])
    
    corpus_length = len(corpus)
    vocabulary_length = len(vocabulary)
    
    def idf_computing(df):
        return np.log10(corpus_length / df)

    vectorized_idf = np.vectorize(idf_computing)
    
    inverse_vocabulary = {v: k for k, v in dict(enumerate(vocabulary)).items()}
    
    df = np.zeros(vocabulary_length, dtype=int)
    
    for document in corpus:
        document_vector = np.zeros(vocabulary_length, dtype=int)
        for word in document.split(' '):
            if word != '':
                document_vector[inverse_vocabulary[word]] += 1
                
        for word in np.unique(document.split(' ')):
            if word != '':
                df[inverse_vocabulary[word]] += 1
        
        tf.append(document_vector)
        
    idf = vectorized_idf(df)
    
    tf = [tf_elem * idf for tf_elem in tf]
    
    tf = np.stack(tf, axis=0 )
    
    return tf

In [17]:
tfidf_weight = tf_idf_vectorizer(wiki['lemmatized_document'])

In [18]:
nn_cosine = NearestNeighbors(metric='cosine')
nn_cosine.fit(tfidf_weight)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [19]:
facts_indexes = [563, 564, 565]

In [20]:
def get_nearest_documents(index):
    cosine, indices = nn_cosine.kneighbors(tfidf_weight[index].reshape(1, -1), n_neighbors = 11)

    neighbors_cosine = pd.DataFrame({'cosine': cosine.flatten(), 'id': indices.flatten()})

    nearest_documents = (wiki.\
                    merge(neighbors_cosine, right_on = 'id', left_index = True).\
                    sort_values('cosine')[['id', 'document', 'cosine']])

    return nearest_documents

def print_results_for_document(index):
    nearest_documents = get_nearest_documents(index)
    documents = nearest_documents['document'].values
    cosines = nearest_documents['cosine'].values
    
    fact = documents[0]
    
    printmd("Факт: " + "**" + fact + "**")
    printmd("Близкие документы:")
    
    for doc, cosine in zip(documents[1:], cosines[1:]):
        printmd(f"* [{cosine:.2f}] {doc}")

In [21]:
print_results_for_document(facts_indexes[0])

Факт: **Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.**

Близкие документы:

* [0.78] В итоге он был оправдан.

* [0.79] После начала Второй мировой войны работал в лазарете Ваффен С  С 1941 года служил врачом в концлагере Бухенвальд, в июне 1942 года был переведён в концлагерь Майданек.

* [0.84] В октябре 1943 года стал главным врачом в концлагере Гросс-Розен, а с сентября 1944 года служил в концлагере Дахау.

...

* [0.91] В 1950 году на Ленинградском фарфоровом заводе была выпущена декоративная ваза с росписью художника Валентина Сарлунда, в основе которой лежала картина Владимира Серова.

* [0.91] 1 УК РФ предусматривается значительное наказание — штраф от 150 до 300 тысяч рублей или лишение свободы сроком до 2 лет.

* [0.92] Эрнст Генрих Шмидт (нем.  Ernst Heinrich Schmidt; 27 марта 1912, Альтенбург, Германская империя — 28 ноября 2000, Целле, Германия) — немецкий врач, гауптштурмфюрер СС, служивший в различных концлагерях.

* [0.92] Из-за отсутствия доказательств 20 марта 1979 года Шмидт вновь был оправдан, а 19 апреля 1979 года освобождён.

* [0.93] Мария Скрипник, работавшая в Совнаркоме по приёму делегаций к Ленину, вспоминала, что временем неудержимого паломничества к Ленину были первые числа ноября 1917 года.

In [22]:
print_results_for_document(facts_indexes[1])

Факт: **Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их тщательно дезинфицировали.**

Близкие документы:

* [0.89] Сам Серов утверждал, что этот персонаж самовольно оставил окопы мировой войны, фронт стал для него настоящей революционной школой, к Ленину он пришёл, будучи избран для этой миссии своими земляками.

* [0.89] Советский, а после эмиграции — британский историк искусства Игорь Голомшток утверждал, что «обязательной принадлежностью колхозных клубов и сельсоветов было изображение типа „Ходоки у Ленина“  Серова».

* [0.89] В своей версии сцены приёма ходоков Базелиц полностью удаляет фигуру Ленина, сосредочившись на тяжёлом положении крестьян.

...

* [0.90] Крестьянин, затаив дыхание, «с надеждой и любовью безотрывно смотрит на вождя».

* [0.90] Владимир Серов утверждал, что ленинская тема «захватила его всего».

* [0.91] В этой обстановке в деревне возникали сложные проблемы, вызывавшие потребность личного общения крестьян с председателем рабоче-крестьянского правительства.

* [0.92] После того, как   Калинин был избран председателем ВЦИКа, Владимир Ильич передал ему личное общение с крестьянами, так как «Михаил Иванович лучше других знает деревню и поэтому он спокоен за то, что каждое ходатайство крестьян будет рассмотрено подробно и хорошо».

* [0.93] Второе направление — обобщённо-монументальный подход, представлявший Ленина революционером, трибуном и вождём.

In [23]:
print_results_for_document(facts_indexes[2])

Факт: **В России акции протеста проходят не только на площадях, но и на поездах.**

Близкие документы:

* [0.84] Одна такая акция имела место 30 июля 2010 года на Горьковском направлении МЖД в день запуска скоростного поезда «Сапсан» сообщением Москва — Нижний Новгород, одной из заявленных целей которой, помимо приветствия «Сапсана», являлся протест против отмены пригородных электричек, обусловленной ремонтом путей в преддверии запуска скоростного поезда).

* [0.85] На многих миниатюрных железных дорогах, которые могут использоваться для катания людей, езда снаружи (верхом) является не только разрешённым, но и единственно возможным способом проезда.

* [0.85] Глядя на них, художник думал о дорогах, по которым они прошли.

...

* [0.88] По мнению практикующих трейнсёрферов, проезд снаружи вагона может быть безопасным, но только при стечении ряда внешних обстоятельств и выполнении зацепером определённых правил предосторожности.

* [0.91] В странах Западной и Центральной Европы езда на грузовых поездах с их внешней стороны также имеет некоторое распространение, но в меньшей степени.

* [0.91] Дезинфекция полностью может их и не уничтожить, но уменьшает количество микроорганизмов до приемлемого уровня.

* [0.91] В композиции нельзя ничего переместить, так как это не только нарушило бы заполненность холста в чисто формальном плане (это всегда принимает во внимание любой художник), но и нарушило бы единство содержания картины, которое художник раскрыл в деталях её композиционного строя.

* [0.91] Только после этого он подбирал типажи и делал эскизы.